# Hysteresis Data Analysis with RockmagPy

In this notebook, we will analyze hysteresis loop data using `rockmagpy`. Based on the quantitative methods of **Jackson and Solheid (2010)**, we will process raw measurements, correct for paramagnetic slopes, and calculate critical parameters with quality control.

**Key Concepts:**

* **Hysteresis Loops:** Measuring magnetization ($M$) vs. applied field ($B$).

* **Hysteresis Parameters:**
    * **$M_s$ (Saturation Magnetization):** The maximum ferromagnetic moment, calculated by fitting high-field data to remove the paramagnetic contribution.
    * **$M_r$ (Saturation Remanence):** The magnetization remaining at zero field ($B=0$); the vertical axis intercept.
    * **$B_c$ (Coercivity):** The reverse field required to reduce magnetization to zero ($M=0$); the horizontal axis intercept.

* **Quality Factor ($Q$):** A signal-to-noise ratio defined by Jackson and Solheid (2010).
    * It uses loop symmetry to distinguish physical signal from random noise.
    * **High $Q$** values indicate a well-defined loop.
    * **Low $Q$** values suggest the data may need numerical filtering or should be rejected.

## Sample Context: The East-Central Minnesota Batholith (ECMB)

The samples analyzed in this notebook come from the **East-Central Minnesota Batholith (ECMB)**, a suite of post-orogenic plutons emplaced ca. 1,780 Ma following the Penokean orogeny. These samples specifically target northeast-trending diabase dikes that intrude the granitic and granodioritic units of the batholith.

### Study Summary
A recent paleomagnetic study of these dikes (*Swanson-Hysell et al., 2021*) established a new paleomagnetic pole for the Superior province.

* **Key Result:** The study confirmed that by ca. 1,780 Ma, the Superior province was coherent with the rest of Laurentia (including the Slave and Rae provinces) following the Trans-Hudson orogeny.
* **Significance:** This result supports the "NENA" (Northern Europe and North America) connection, placing Laurentia and Baltica together in the supercontinent Nuna.
* **Magnetic Mineralogy:** The primary remanence in these dikes is held by low-Ti titanomagnetite (magnetite). However, some dikes experienced hydrothermal alteration associated with the much younger (~1.1 Ga) Midcontinent Rift, which formed secondary monoclinic pyrrhotite.

### Specimen Groupings & Magnetic Behavior
The samples we are analyzing display different magnetic behaviors based on the preservation of their primary magnetite and the extent of secondary alteration (e.g., pyrrhotite growth).

**1. Mid-Coercivity Dominated (Magnetite)**
*Remanence is dominated by a mid-coercivity component with only a very minor high-coercivity component. These were interpreted to represent the best-preserved primary signal.*
* `NED34-1c`
* `NED35-2c`
* `NED1-8c`
* `NED2-1c`
* `NED5-8c`

**2. Mixed Coercivity (Magnetite + Minor Pyrrhotite)**
*Remanence contains a mid-coercivity component but shows a distinct, though minor, high-coercivity component.*
* `NED11-2c`
* `NED12-7c`
* `NED13-3c`
* `NED17-3c`

**3. High-Coercivity Dominated (Pyrrhotite/Alteration)**
*Remanence is dominated by a high-coercivity component, indicative of secondary pyrrhotite growth which obscures the primary signal.*
* `NED4-5c`
* `NED4-7c`
* `NED36-7c`
* `NED17-1c`

## Install PmagPy

First, we install and import the necessary libraries.

In [ ]:
!pip install --upgrade pmagpy

In [ ]:
import pmagpy.ipmag as ipmag
import pmagpy.pmag as pmag
import pmagpy.rockmag as rmag
import pmagpy.contribution_builder as cb

import os
import requests
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from bokeh.plotting import figure, show
from bokeh.io import output_notebook
output_notebook()

## Data Import and Preparation

We will be working with the file `ECMB.TXT`, which is formatted according to the **MagIC (Magnetics Information Consortium)** standard. This file contains raw measurement data from the hysteresis experiments we made in class.

The code below performs the following steps:
1.  **Download:** Retrieves the example dataset.
2.  **Unpack:** Uses `pmag.magic_read` to parse the MagIC format into Python dictionaries and lists.
3.  **DataFrame Creation:** Converts the raw data into a Pandas DataFrame for easier manipulation.

In [ ]:
# 1. Define the URL and local filename
url = "https://raw.githubusercontent.com/Institute-for-Rock-Magnetism/2026_ESCI_pmag_course/refs/heads/main/W3_anisotropy_hysteresis/data/ECMB.TXT"
filename = "ECMB.TXT"

# 2. Download the file
print(f"Downloading {filename}...")
response = requests.get(url)
with open(filename, 'wb') as f:
    f.write(response.content)
print("Download complete.")

# 3. Unpack the MagIC contribution
# This extracts 'measurements.txt', 'specimens.txt', etc. into the current directory
print("Unpacking MagIC contribution...")
ipmag.download_magic(filename)
print("Unpacking complete. Check your directory for 'measurements.txt'.")

In [ ]:
# set the dir_path to the directory where the measurements.txt file is located
dir_path = '/content/'

# create a contribution object from the tables in the directory
contribution = cb.Contribution(dir_path)
measurements = contribution.tables['measurements'].df
specimens = contribution.tables['specimens'].df

In [ ]:
hyst_measurements = measurements[measurements['method_codes'] == 'LP-HYS']

In [ ]:
hyst_experiments = rmag.make_experiment_df(hyst_measurements)
hyst_experiments.head()

## Data Cleaning

Before proceeding with the analysis, we must filter out specimen `NED23-8c`.

**Reason for Exclusion:**
* **Mass Error:** This specific specimen has an incorrect/missing mass measurement recorded in the raw data.
* **Impact:** Since magnetization is normalized by mass (e.g., $Am^2/kg$), an error in the mass value will propagate to all specific magnetization parameters ($M_s$, $M_r$), making them invalid for comparison.

In [ ]:
hyst_experiments = hyst_experiments[hyst_experiments['specimen'] != 'NED23-8c']
hyst_experiments = hyst_experiments.reset_index(drop=True)
hyst_experiments

## Hysteresis Loop Processing

Now that the data is loaded, we process each specimen to extract the hysteresis parameters.

To analyze a specific loop, we must identify it using two key identifiers:
* **Experiment Name:** The unique ID for the measurement sequence (e.g., `'IRM-VSM3-LP-HYS-249020'`). This filters the raw measurements DataFrame to get the specific data points for one loop.
* **Specimen Name:** The label for the physical sample (e.g., `'NED2-1c'`). This is passed to the processing function to label the output plots and results.

The processing steps typically involve:
1.  **Paramagnetic Correction:** High-field slopes (dominated by paramagnetic minerals) are subtracted to isolate the ferromagnetic signal.
2.  **Parameter Extraction:** The code calculates $M_s$, $M_r$, and $H_c$ from the corrected loops.
3.  **Loop Quality:** A "quality factor" or shape parameter is often calculated to determine if the loop is sufficiently closed and saturated.

In the code below, observe how we define both the `experiment_name` to filter the data and the `specimen_name` for the analysis function.

In [ ]:
#specify the experiment name and specimen name
experiment_name = 'IRM-VSM3-LP-HYS-218868'
specimen_name = 'NED23-8c'

experiment_hyst = measurements[measurements['experiment'] == experiment_name].reset_index(drop=True)
experiment_results = rmag.process_hyst_loop(experiment_hyst['meas_field_dc'].values,
                                            experiment_hyst['magn_mass'].values,
                                            specimen_name, show)
NED1_5c_hyst_process_result = experiment_results
NED1_5c_hyst_process_result['plot']

In [ ]:
results = rmag.process_hyst_loops(hyst_experiments,measurements,
                                  show_results_table=False, show_plots=False)

In [ ]:
results.head()

In [ ]:
results['Ms']

In [ ]:
results.loc['NED18-2c', 'Ms']

In [ ]:
# Define the classification mapping based on the study groupings
classification_map = {
    # Mid-Coercivity Dominated (Magnetite)
    'NED34-1c': 'Mid-Coercivity',
    'NED34-6c': 'Mid-Coercivity',
    'NED35-2c': 'Mid-Coercivity',
    'NED1-5c':  'Mid-Coercivity',
    'NED1-8c':  'Mid-Coercivity',
    'NED2-1c':  'Mid-Coercivity',
    'NED2-8c':  'Mid-Coercivity',
    'NED5-8c':  'Mid-Coercivity',

    # Mixed Coercivity (Magnetite + Minor Pyrrhotite)
    'NED11-2c': 'Mixed',
    'NED12-7c': 'Mixed',
    'NED13-3c': 'Mixed',
    'NED17-3c': 'Mixed',

    # High-Coercivity Dominated (Pyrrhotite/Alteration)
    'NED4-1c':  'High-Coercivity',
    'NED4-5c':  'High-Coercivity',
    'NED4-7c':  'High-Coercivity',
    'NED36-7c': 'High-Coercivity',
    'NED17-1c': 'High-Coercivity'
}

# Create a new column 'classification' by mapping the index (specimen name) to the dictionary
# Note: This assumes your results DataFrame has the specimen names as the index
results['classification'] = results.index.map(classification_map)

# Display the first few rows to verify
results[['classification', 'Ms', 'Q']]

## Exercise: Data Interpretation

Now that we have processed the hysteresis data, perform the following analysis in new code cells:

1.  **Filter by Quality:**
    * Filter the results DataFrame to include only specimens where the shape parameter $Q$ is acceptable (typically $Q > 2.0$ suggests a high-quality loop).

2.  **Saturation Magnetization ($M_s$) Analysis:**
    * Create a plot of $M_s$ for each valid specimen.
    * **Estimation:** Assuming the dominant magnetic mineral is **magnetite** (which has a theoretical $M_s$ of approx. $92 \ Am^2/kg$), estimate the concentration (weight percent) of magnetite in each specimen using the measured $M_s$.
    * *Hint: Concentration % = (Measured $M_s$ / 92) * 100*

3.  **Squareness vs. Coercivity:**
    * Calculate the "Squareness" ($M_r / M_s$) for the filtered data.
    * Create a scatter plot comparing "Squareness" ($M_r / M_s$) on the y-axis versus Coercivity ($H_c$) on the x-axis.
    * Write out a caption that describes how these values were obtrained and that discusses what the values imply about the magnetic domain state of the samples that are dominated by magnetite (i.e. the ones categorized as Mid-Coercivity).